Experimento 8 - KNN Imputer, MAD outlier detection, Z-Score normalization, MLP classifier

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
!pip install --quiet ucimlrepo
from ucimlrepo import fetch_ucirepo

In [2]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [3]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [4]:
import random

#Introducing Missing values (5%)
x = df.copy()

for index, i in enumerate(df):
  for jndex, j in enumerate(df[i]):
    if random.randint(0,100) < 5:
      x[i][jndex] = np.NaN

df = x

<ipython-input-4-fbb7389abead>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i][jndex] = np.NaN


In [5]:
df.isna().sum()

Area               711
Perimeter          692
MajorAxisLength    696
MinorAxisLength    680
AspectRatio        685
Eccentricity       683
ConvexArea         655
EquivDiameter      667
Extent             709
Solidity           678
Roundness          697
Compactness        689
ShapeFactor1       670
ShapeFactor2       665
ShapeFactor3       658
ShapeFactor4       671
dtype: int64

In [6]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,NaN,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,NaN,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,NaN,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,NaN,0.861794,0.994199
4,30140.0,620.134,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,42494.0,NaN,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [7]:
#Imputing Missing Values with KNN Imputer
from sklearn.impute import KNNImputer as knni

imputer = knni(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
df_imputed

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.2910,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.769425,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,627.4318,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.1100,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008.0,645.8840,210.557999,182.516516,1.153638,0.498616,31331.4,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.002979,0.861794,0.994199
4,30140.0,620.1340,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.6960,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,757.4990,281.576392,190.713136,1.476439,0.735702,42494.0,233.362591,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42139.0,759.3210,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.7790,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [8]:
#adding the labels
df_imputed['Class'] = targets
df_pre_missing_values = df.copy()
print(df_imputed.groupby('Class').agg(['count']))
df_imputed['Class'] = df_imputed['Class'].transform(lambda x: 0 if x == 'BARBUNYA' else (1 if x == 'BOMBAY' else (2 if x == 'CALI' else (3 if x == 'DERMASON' else (4 if x == 'HOROZ' else (5 if x == 'SEKER' else 6))))))

          Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count     count           count           count       count   
Class                                                                  
BARBUNYA  1322      1322            1322            1322        1322   
BOMBAY     522       522             522             522         522   
CALI      1630      1630            1630            1630        1630   
DERMASON  3546      3546            3546            3546        3546   
HOROZ     1928      1928            1928            1928        1928   
SEKER     2027      2027            2027            2027        2027   
SIRA      2636      2636            2636            2636        2636   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                                      
BARBUNYA         1322       1322          1322   13

In [9]:
#Outlier Removal with MAD
from scipy import stats

print(f'Pre Outlier Shape: {df_imputed.shape}')

df_no_outliers = df_imputed.copy()
print(df_no_outliers['Class'].unique())
for i in df_no_outliers['Class'].unique():
    class_unique = df_no_outliers[df_no_outliers['Class'] == i]
    for feature in class_unique:
      mad = 1.4826 * np.median(np.absolute(class_unique[feature] - class_unique[feature].median()))
      upper = class_unique[feature].median() + (3 * mad)
      lower = class_unique[feature].median() - (3 * mad)
      excluded_lower = pd.Series(class_unique[class_unique[feature] < lower].index)
      excluded_upper = pd.Series(class_unique[class_unique[feature] > upper].index)
      df_no_outliers.drop(excluded_lower.values, inplace = True, errors='ignore')
      df_no_outliers.drop(excluded_upper.values, inplace = True, errors='ignore')


print(f'Pos Outlier Shape: {df_no_outliers.shape}')

Pre Outlier Shape: (13611, 17)
[5 0 1 2 4 6 3]
Pos Outlier Shape: (12035, 17)


In [10]:
df_no_outliers['Class'] = df_no_outliers['Class'].transform(lambda x: 'BARBUNYA' if x == 0 else ('BOMBAY' if x == 1 else ('CALI' if x == 2 else ('DERMASON' if x == 3 else ('HOROZ' if x == 4 else ('SEKER' if x == 5 else 'SIRA'))))))
df_no_outliers = df_no_outliers.reset_index()
print(df_no_outliers.groupby('Class').agg(['count']))

         index  Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count count     count           count           count       count   
Class                                                                        
BARBUNYA  1191  1191      1191            1191            1191        1191   
BOMBAY     437   437       437             437             437         437   
CALI      1510  1510      1510            1510            1510        1510   
DERMASON  3185  3185      3185            3185            3185        3185   
HOROZ     1588  1588      1588            1588            1588        1588   
SEKER     1743  1743      1743            1743            1743        1743   
SIRA      2381  2381      2381            2381            2381        2381   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                              

In [11]:
df_no_outliers = df_no_outliers.drop(['index'], axis='columns')
df_no_outliers

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395.0,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.769425,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
2,30279.0,634.927,212.560556,181.510182,1.171067,0.520401,30600.0,196.510484,0.775688,0.989510,0.943852,0.923726,0.007020,0.003153,0.853270,0.999236,SEKER
3,30519.0,629.727,212.996755,182.737204,1.165591,0.513760,30938.4,197.124320,0.770682,0.989367,0.967109,0.925480,0.006979,0.003158,0.856514,0.998345,SEKER
4,30685.0,635.681,213.534145,183.157146,1.165852,0.514081,31044.0,197.659696,0.771561,0.988436,0.954240,0.925658,0.006959,0.003152,0.856844,0.998953,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12030,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
12031,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,42494.0,233.362591,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
12032,42139.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
12033,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [12]:
#Normalization with Z-Score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(df_no_outliers[cols])
df_scaled = pd.DataFrame(scaler.transform(df_no_outliers[cols]), columns = cols)

df_scaled['Class'] = df_no_outliers['Class']
df_scaled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,-0.858236,-1.162888,-1.330318,-0.643596,-1.586858,-2.270522,-0.857165,-1.084259,0.397578,0.313487,1.432132,1.877868,0.702244,2.439644,1.976108,0.968995,SEKER
1,-0.823747,-1.096521,-1.275029,-0.596841,-1.535499,-2.125330,-0.823588,-1.027817,0.579068,0.518047,1.252914,1.802126,0.622745,2.270190,1.889688,1.075537,SEKER
2,-0.792269,-1.044572,-1.278188,-0.469125,-1.694015,-2.601166,-0.792249,-0.974314,0.528418,0.503793,1.182519,2.049201,0.419587,2.448989,2.173201,1.128348,SEKER
3,-0.783866,-1.069545,-1.273000,-0.441036,-1.716477,-2.675826,-0.780595,-0.963718,0.423836,0.462190,1.592058,2.078192,0.382455,2.458360,2.206772,0.850909,SEKER
4,-0.778053,-1.040951,-1.266607,-0.431423,-1.715404,-2.672213,-0.776959,-0.954477,0.442211,0.191169,1.365441,2.081136,0.364084,2.446897,2.210184,1.040311,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12030,-0.378472,-0.445358,-0.372245,-0.367610,-0.128534,0.148646,-0.382162,-0.370073,-0.748176,0.742890,0.702701,0.035470,0.272940,0.059776,-0.002708,0.863575,DERMASON
12031,-0.378332,-0.455909,-0.457238,-0.258450,-0.441452,-0.180744,-0.382644,-0.338195,1.035063,0.865259,0.798004,0.372363,0.118318,0.292523,0.339910,0.811714,DERMASON
12032,-0.377001,-0.447159,-0.457672,-0.247580,-0.457276,-0.199152,-0.380061,-0.368080,-0.427365,0.617006,0.734766,0.380254,0.112059,0.296670,0.348038,0.359893,DERMASON
12033,-0.376721,-0.425749,-0.435753,-0.268464,-0.388594,-0.120571,-0.376686,-0.367701,-0.940042,0.009778,0.549562,0.293131,0.150589,0.234975,0.258558,-0.121020,DERMASON


In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate


x_train = df_scaled.iloc[:, 0:16]
y_train = df_scaled.iloc[:, 16]
classifier = MLPClassifier(activation='logistic', solver='adam', alpha=1e-5, hidden_layer_sizes=(12, 3), random_state=1, verbose=True, learning_rate_init=0.3, tol=1e-3, max_iter=500)
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}


y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)
print('Experimento 8')
print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')


Iteration 1, loss = 1.18307261
Iteration 2, loss = 0.57708767
Iteration 3, loss = 0.38865165
Iteration 4, loss = 0.36720932
Iteration 5, loss = 0.26188146
Iteration 6, loss = 0.23804377
Iteration 7, loss = 0.21908430
Iteration 8, loss = 0.19997317
Iteration 9, loss = 0.18834701
Iteration 10, loss = 0.18963309
Iteration 11, loss = 0.18718233
Iteration 12, loss = 0.18552321
Iteration 13, loss = 0.18582311
Iteration 14, loss = 0.17643597
Iteration 15, loss = 0.16431088
Iteration 16, loss = 0.19127362
Iteration 17, loss = 0.16621434
Iteration 18, loss = 0.16066862
Iteration 19, loss = 0.16434541
Iteration 20, loss = 0.17763737
Iteration 21, loss = 0.18681594
Iteration 22, loss = 0.17299389
Iteration 23, loss = 0.15611767
Iteration 24, loss = 0.16151175
Iteration 25, loss = 0.18775580
Iteration 26, loss = 0.18735386
Iteration 27, loss = 0.16518791
Iteration 28, loss = 0.17102282
Iteration 29, loss = 0.17308410
Iteration 30, loss = 0.16686012
Iteration 31, loss = 0.17048521
Iteration 32, los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Iteration 1, loss = 1.23928130
Iteration 2, loss = 0.74661266
Iteration 3, loss = 0.48970608
Iteration 4, loss = 0.41857547
Iteration 5, loss = 0.39716422
Iteration 6, loss = 0.33392191
Iteration 7, loss = 0.28567530
Iteration 8, loss = 0.29560125
Iteration 9, loss = 0.27374373
Iteration 10, loss = 0.27201813
Iteration 11, loss = 0.28470848
Iteration 12, loss = 0.27792601
Iteration 13, loss = 0.25797256
Iteration 14, loss = 0.24415846
Iteration 15, loss = 0.24482489
Iteration 16, loss = 0.23518053
Iteration 17, loss = 0.22923306
Iteration 18, loss = 0.26344187
Iteration 19, loss = 0.23090059
Iteration 20, loss = 0.27581385
Iteration 21, loss = 0.23915883
Iteration 22, loss = 0.25038055
Iteration 23, loss = 0.24509938
Iteration 24, loss = 0.23309456
Iteration 25, loss = 0.21002578
Iteration 26, loss = 0.22548475
Iteration 27, loss = 0.23221364
Iteration 28, loss = 0.23156579
Iteration 29, loss = 0.23581811
Iteration 30, loss = 0.23590566
Iteration 31, loss = 0.22116243
Iteration 32, los